In [1]:
import tensorflow as tf

In [2]:
import glob

In [3]:
from itertools import groupby
from collections import defaultdict

In [4]:
image_filenames = glob.glob("./imagenet-dogs/n02*/*.jpg")
image_filenames[0:2]

['./imagenet-dogs/n02097658-silky_terrier/n02097658_26.jpg',
 './imagenet-dogs/n02097658-silky_terrier/n02097658_4869.jpg']

In [5]:
training_dataset = defaultdict(list)
testing_dataset = defaultdict(list)

In [6]:
image_filename_with_breed = list(map(lambda filename: (filename.split("/")[2],filename),image_filenames))

In [10]:
for dog_breed,breed_images in groupby(image_filename_with_breed,lambda x : x[0]):
    for i, breed_image in enumerate(breed_images):
        if i % 5 == 0:
            testing_dataset[dog_breed].append(breed_image[1])
        else:
            training_dataset[dog_breed].append(breed_image[1])
    breed_training_count = len(training_dataset[dog_breed])
    breed_testing_count = len(testing_dataset[dog_breed])
    
    assert round(breed_training_count / (breed_training_count + breed_testing_count),2) > 0.18, "Not enough testing images."

In [ ]:
def write_records_file(dataset,record_location)